In [ ]:
import pandas as pd
from datetime import datetime as dt
from datetime import timedelta
import xlsxwriter
import numpy as np
import pyodbc
with open('login.txt') as key:
    login=key.read().strip()
    
cnxn = pyodbc.connect(login)

sql_query="""
select pmnum, lead, person.displayname, pm.description, left(asset.location,3) 'Building', locations.room,
laststartdate, lastcompdate, frequency, frequnit, 
sunday, monday, tuesday, wednesday, thursday, friday, saturday, sunday, pm.vendor

from pm
full join asset on asset.assetnum = pm.assetnum
full join locations on asset.location = locations.location
full join person on person.personid = pm.[lead]

where pm.siteid='BOSTON'
and pm.status = 'ACTIVE'
and (lead <> 'SSANKU01' or lead is null)
"""

#read the sql query into a df
data = pd.read_sql(sql_query,cnxn, parse_dates='Start Date')

#clean data - fill null values for unassigned PMs as well as PMs assigned to a vendor
#this happens for both lead and displayname columns
data.loc[data['lead'].isna(),'lead'] = 'Not Assigned'
data.loc[data['lead'] == 'Not Assigned','displayname'] = 'Not Assigned'
data.loc[pd.notnull(data['vendor']),'lead'] = 'Vendor'
data.loc[data['lead'] == 'Vendor','displayname'] = 'Vendor'

In [ ]:
#Function add months to a date. i.e. 1/15/20 will become 3/15/20. Does not add 30 days, but adds 1 to the month. 
# the native pandas function does not do this well.

def add_months(input_date,months_to_add):
    iyear = input_date.year
    imonth = input_date.month + int(months_to_add)
    if imonth>12:
        iyear=iyear + int(imonth//12)
        if imonth%12==0:
            imonth=1
        else:
            imonth=imonth%12
    iday = input_date.day
    
    if imonth==2 and iday>28:
        new_date = dt(year=iyear, month=imonth, day=28).date()
    elif imonth in [9, 4, 6, 11] and iday==31: 
        new_date = dt(year=iyear, month=imonth, day=30).date()
    else:
        new_date = dt(year=iyear, month=imonth, day=iday).date()
    new_date = dt(new_date.year, new_date.month, new_date.day)
    return new_date

In [ ]:
# Function to forecast the WOs that will be generated from PMs for the next year

# test variables for testing the function
# testdate = dt.strptime('2020-04-02','%Y-%m-%d')
# new_date= testdate
# test_frequency = 1
# test_frequnit = 'DAYS'
# test_frequnit = 'WEEKS'
# test_frequnit = 'YEARS'
# test_frequnit = 'MONTHS'
# test_tech = 'SMACHA01'

def fill_data (pm,tech,frequency,frequnit,last_start_date):
    todays_date = dt.today()
    instance_number = 0
    new_date = last_start_date
    
    if pd.isnull(new_date):
        pass
    while new_date < (todays_date + timedelta(days=365)):
        instance_number += frequency
        if instance_number > 365:
            break
            
        if frequnit == 'DAYS':
            if frequency==30:
                new_date = new_date + timedelta (days=frequency)
            else:
                break
            if new_date > (todays_date+timedelta (days=365)):
                break
            
        elif frequnit == 'WEEKS':
            new_date = new_date + timedelta (days=7*frequency)
            if new_date > (todays_date+timedelta (days=365)):
                break
        elif frequnit == 'MONTHS':
            new_date = add_months(last_start_date,instance_number) 
            if new_date > (todays_date+timedelta (days=365)):
                break
        elif frequnit == 'YEARS':
            new_date = add_months(last_start_date,(instance_number*12)) 
            if new_date > (todays_date+timedelta (days=365)):
                break      
#         print (pm, instance_number, new_date)
        
        pm_month = new_date.strftime("%b")
        pms_per_month.loc[tech,pm_month] += 1
    
# fill_data(test_tech, test_frequency, test_frequnit, testdate)

In [ ]:
#create a new dataframe of the monthly totals, index are techs, cols are months
techs = data['lead'].unique().tolist()
techs.sort()
cols = ['name','Jan', 'Feb', 'Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
pms_per_month = pd.DataFrame(columns = cols, index = techs)
pms_per_month.fillna(0, inplace=True)
# display(pms_per_month)

#add the full name of the tech to the pms_per_tech dataframe
for tech in techs:
    full_name = data[data['lead']==tech]['displayname'].unique()
    pms_per_month.loc[tech,'name'] = full_name


In [ ]:
data [data['laststartdate'].notnull()].apply(lambda x: fill_data(x.pmnum, x.lead, x.frequency, x.frequnit, x.laststartdate), axis=1)
x=1

# add a total column
pms_per_month['Total']=pms_per_month.iloc[:,1:].sum(axis=1).astype('int')

In [ ]:
pms_per_month

In [ ]:
def get_answers(question,choices):
  choice = ""
  while choice.lower() not in choices:
        
        choice = input(question + " Choose [%s]:" % ", ".join(choices))
        choice = choice.lower()
  return choice

file_name = dt.today().strftime('%Y.%m.%d')+  " PMs per Tech per Month Report.xlsx"
print (file_name)
export_file=get_answers("Export file?",['y', 'n'])
if export_file == "y":
    pms_per_month.to_excel(file_name)


In [ ]:
technician = input ("Tech UTLN?")
# one_techs_pms = data[data['lead']=='MDMECH01']
one_techs_pms = data[data['lead']==technician]

file_name = dt.today().strftime('%Y.%m.%d')+  " unassigned PMs.xlsx"
print (file_name)
export_file=get_answers("Export file?",['y', 'n'])
if export_file == "y":
    one_techs_pms.iloc[:,0:10].to_excel(file_name, index=False)

In [ ]:
print (one_techs_pms.iloc[:,0:10].to_string(index=False))